In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.model_selection import KFold
from keras.models import Model, Sequential
from keras import layers
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import  Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, GRU, Reshape, LSTM, TimeDistributed, SimpleRNN
from random import shuffle
import math
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from keras.models import load_model
import statistics
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import sys
import os
from datetime import time

In [ ]:
from ipynb.fs.full.DatasetManage import read_and_store_data
from ipynb.fs.full.FeatureExtraction import feature_extraction

Configuration parameter

In [ ]:
PathSpectogramFolder='Spectogram'

interictalSpectograms=[]
preictalSpectograms=[]  #This array contains syntetic data, it's created to have a balance dataset and it's used for training
preictalRealSpectograms=[]  #This array containt the real preictal data, it's used for testing
filesPath=[]

patients = ["01","02","03","05","09","10","23"]
nSeizure=0
OutputPathModels = 'models/'

initial_learning_rate = 0.001

dir_name = "model_checkpoint"

Parameter

In [ ]:
perfInd = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'MCC', 'Kappa', 'Time']
channels = ['FP1-F7', 'F7-T7','T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'seizure']

dataset = 'CHB_MIT'

feature_extracted = 'Features.csv'

sample_rate = 256
time_window = 2
step = time_window * sample_rate

test_ratio = 0.3

pca_tolerance = 0.9

undersampling_rate = 0.2

oversampling_neighbors = 11

k_fold = 5

batch = 10
epochs = 100
dropout_percentage = 0.2
loss_function = 'mean_squared_error'
metric = 'accuracy'

In [ ]:
def optimizer_SGD (initial_learning_rate, decay_steps, decay_rate):
    # We define the optimizer with an initial learning rate
    optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

    # We define the larning rate schedule with an exponential decay, specifying the number of decay steps and the decay rate
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps, decay_rate, staircase=True)
    
    return optimizer, lr_schedule

In [ ]:
def loadSpectogramData(indexPat):
    global interictalSpectograms
    global preictalSpectograms
    global preictalRealSpectograms
    global nSeizure
    f = open(PathSpectogramFolder+'/paz'+patients[indexPat]+'/legendAllData.txt', 'r')
    line=f.readline()
    while(not "SEIZURE" in line):
        line=f.readline()
    nSeizure=int(line.split(":")[1].strip())
    line=f.readline()
    line=f.readline()#legge il numero di spectogrammi. non lo salvo dato che non mi serve
    nSpectograms=int(line.strip())
    nFileForSeizure=math.ceil(math.ceil(nSpectograms/50)/nSeizure)
    line=f.readline()#leggo il percorso del primo file
    
    #Lettura path files Interictal
    cont=-1
    indFilePathRead=0
    while("npy" in line and indFilePathRead<nSeizure*nFileForSeizure):
        if(indFilePathRead%nFileForSeizure==0):
            interictalSpectograms.append([])
            cont=cont+1
            interictalSpectograms[cont].append(line.split(' ')[2].rstrip())#.rstrip() remove \n
            indFilePathRead=indFilePathRead+1
        else:
            if(len(line.split(' '))>=3):
                interictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead=indFilePathRead+1
            
        line=f.readline()
    line=f.readline()#leggo PREICTAL
    line=f.readline()#leggo n° spectogram
    line=f.readline()#leggo n°seizure(SEIZURE X)

    #Lettura path files Preictal
    cont=-1
    indFilePathRead=0   
    #while(line and indFilePathRead<nSeizure*nFileForSeizure):    
    while(line.strip()!=""):
        if("SEIZURE" in line):
            line=f.readline()#ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            if(len(line.split(' '))>=3):
                preictalSpectograms.append([])
                cont=cont+1
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
                indFilePathRead=indFilePathRead+1
        else:
            if(len(line.split(' '))>=3):
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead=indFilePathRead+1
            
        line=f.readline()
        
    line=f.readline()#leggo REAL_PREICTAL
    line=f.readline()#leggo n° spectogram
    line=f.readline()#leggo n°seizure(SEIZURE X)

    #Lettura path files Real Preictal
    cont=-1
    while(line):
        if("SEIZURE" in line):
            line=f.readline()#ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            preictalRealSpectograms.append([])
            cont=cont+1
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
        else:
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
            
        line=f.readline()
    f.close()

In [ ]:
def getFilesPathWithoutSeizure(indexSeizure):
    for i in range(0, nSeizure):
        if(i!=indexSeizure):
            filesPath.extend(interictalSpectograms[i])
            filesPath.extend(preictalSpectograms[i])
    shuffle(filesPath)
    return filesPath

Create array for training

In [ ]:
def generate_arrays_for_training(paths, start=0, end=100):
    while True:
        from_=int(len(paths)/100*start)
        to_=int(len(paths)/100*end)
        for i in range(from_, int(to_)):
            f=paths[i]
            x = np.load(PathSpectogramFolder+f)
            x=np.array([x])
            x=x.swapaxes(0,1)
            x=x[0]
            if('P' in f):
                y = np.repeat([[0,1]], x.shape[0], axis=0)
            else:
                y = np.repeat([[1,0]], x.shape[0], axis=0)
            yield(x,y)

Create array for prediction

In [ ]:
def generate_arrays_for_predict(indexPat, paths, start=0, end=100):
    while True:
        f = open(PathSpectogramFolder+'/paz'+patients[indexPat]+'/legendAllData.txt', 'r')
        line=f.readline()
        while(not "SEIZURE" in line):
            line=f.readline()
        nSeizure=int(line.split(":")[1].strip())
        from_=int(len(paths)/100*start)
        to_=int(len(paths)/100*end)
        for i in range(from_, int(to_)):
            f=paths[i]
            x = np.load(PathSpectogramFolder+f)
            x=np.array([x])
            x=x.swapaxes(0,1)
            x=x[0]
            yield(x)

Initialization

In [ ]:
def init():
    interictalSpectograms=[]
    preictalSpectograms=[]  # This array contains syntetic data, it's created to have a balance dataset and it's used for training
    preictalRealSpectograms=[]  #This array containt the real preictal data, it's used for testing
    filesPath=[]
    nSeizure=0

    print("START")
    if not os.path.exists(OutputPathModels):
        os.makedirs(OutputPathModels)
    print("Parameters loaded")

    for indexPat in range(0, len(patients)):
        loadSpectogramData(indexPat) 
        filesPath = getFilesPathWithoutSeizure(indexPat)


Create Training and Test Data

In [ ]:
def trainTestData (features, test_ratio, k_fold, perfInd):
    x = features.loc[:, features.columns != 'seizure']
    y = features['seizure']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_ratio, shuffle = True)
    results = pd.DataFrame(columns = perfInd)
    kf = KFold(n_splits = k_fold, shuffle = True)
    return x_train, x_test, y_train, y_test, results, kf

# Here we define a method to split the dataset into training and testing sets, also returning the train/test
# indexes for splitting the dataset into K folds for the K-fold cross validation

def trainTestData_k_fold(features, test_ratio, k_fold):
    x = features.loc[:, features.columns != 'seizure']
    y = features['seizure']
    x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size = test_ratio, shuffle = True, random_state=42)
    kf = KFold(n_splits = k_fold, shuffle = True)
    x_train = np.reshape(x_tr.values, (x_tr.shape[0], 1, x_tr.shape[1]))
    y_train = y_tr.values.astype(int)
    x_test = np.reshape(x_ts.values, (x_ts.shape[0], 1, x_ts.shape[1]))
    y_test = y_ts.values.astype(int)
    return x_train, x_test, y_train, y_test, kf

# Here we define a method to split the dataset into training, validationa and testing sets

def trainTestData_validation (features, test_ratio, val_ratio):
    x = features.loc[:, features.columns != 'seizure']
    y = features['seizure']
    x_1, x_ts, y_1, y_ts = train_test_split(x, y, test_size = test_ratio, random_state=42)
    x_tr, x_v, y_tr, y_v = train_test_split(x_1, y_1, test_size = val_ratio, random_state=42)
    x_train = np.reshape(x_tr.values, (x_tr.shape[0], 1, x_tr.shape[1]))
    y_train = y_tr.values.astype(int)
    x_val = np.reshape(x_ts.values, (x_ts.shape[0], 1, x_ts.shape[1]))
    y_val = y_ts.values.astype(int)
    x_test = np.reshape(x_ts.values, (x_ts.shape[0], 1, x_ts.shape[1]))
    y_test = y_ts.values.astype(int)
    return x_train, x_test, y_train, y_test, x_val, y_val

Plot of the performance of the model

In [ ]:
def plot_performance(train_loss, val_loss, train_acc, val_acc):
    
    epochs = range(len(train_loss))

    plt.figure()
    plt.plot(epochs, train_loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.figure()
    plt.plot(epochs, train_acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.show()


def plot_k_fold (train_loss, train_acc, val_loss, val_acc):
    
    avg_train_loss = np.mean(train_loss, axis=0)
    avg_train_acc = np.mean(train_acc, axis=0)
    avg_val_loss = np.mean(val_loss, axis=0)
    avg_val_acc = np.mean(train_acc, axis=0)

    # Plot delle curve di apprendimento mediate sulle K fold

    epochs = range(1, len(train_loss[0]) + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, avg_train_loss, label='Training loss')
    plt.plot(epochs, avg_val_loss, label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Avg_loss')
    plt.title('Average train and validation loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, avg_train_acc, label='Training accuracy')
    plt.plot(epochs, avg_val_acc, label='Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Avg_accuracy')
    plt.title('Average train and validation accuracy')
    plt.legend()
    plt.show()

def plot_validation (history):
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs = range(len(train_loss))

    plt.figure()
    plt.plot(epochs, train_loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.figure()
    plt.plot(epochs, train_acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.show()

Evaluation of the model, using sensitivity and FPR

In [ ]:
def evaluation_model(model):

    for indexPat in range(0, len(patients)):
        
        loadSpectogramData(indexPat)

        sensResults = []
        FPRResults = []

        result='Patient '+patients[indexPat]+'\n'     
        result='Out Seizure, True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR \n'
        for i in range(0, nSeizure):

            filesPath=interictalSpectograms[i]
            interPrediction=model.predict(generate_arrays_for_predict(indexPat, filesPath), max_queue_size=4, steps=len(filesPath), verbose=0)
            filesPath=preictalRealSpectograms[i]
            preictPrediction=model.predict(generate_arrays_for_predict(indexPat, filesPath), max_queue_size=4, steps=len(filesPath), verbose = 0)
            
            secondsInterictalInTest=len(interictalSpectograms[i])*50*30#50 spectograms for file, 30 seconds for each spectogram
            acc=0#accumulator
            fp=0
            tp=0
            fn=0
            lastTenResult=list()
            
            for el in interPrediction:
                if(el[1]>0.5):
                    acc=acc+1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if(len(lastTenResult)>10):
                    acc=acc-lastTenResult.pop(0)
                if(acc>=4):
                  fp=fp+1
                  lastTenResult=list()
                  acc=0
            
            lastTenResult=list()
            for el in preictPrediction:
                if(el[1]>0.5):
                    acc=acc+1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if(len(lastTenResult)>10):
                    acc=acc-lastTenResult.pop(0)
                if(acc>=4):
                  tp=tp+1 
                else:
                    if(len(lastTenResult)==10):
                       fn=fn+1 
                       
            if (tp+fn)!=0:
                sensitivity=tp/(tp+fn)
            else:
                sensitivity=0
            FPR=fp/(secondsInterictalInTest/(60*60))
            
            result=result+str(i+1)+','+str(tp)+','+str(fp)+','+str(fn)+','+str(secondsInterictalInTest)+','
            result=result+str(sensitivity)+','+str(FPR)+'\n'
            #print('True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR')
            #print(str(tp)+','+str(fp)+','+str(fn)+','+str(secondsInterictalInTest)+','+str(sensitivity)+','+str(FPR))
        
            sensResults.append(sensitivity)
            FPRResults.append(FPR)
  
                    
        sdSENS=round(statistics.stdev(sensResults)*100,2)
        avSENS=round(statistics.mean(sensResults)*100,2)
        
        sdFPR=round(statistics.stdev(FPRResults)*100,2)
        avFPR=round(statistics.mean(FPRResults)*100,2)

        print('Patient '+patients[indexPat]+"  & "+str(avSENS)+" +- "+str(sdSENS)+"   & "+str(avFPR)+" +- "+str(sdFPR))

Read and Store Data

In [ ]:
csvImportFile = 'CHB'
print('Reading data from', csvImportFile)
df = pd.read_csv(csvImportFile, delimiter = ',', header = 0)

Feature Extraction 

In [ ]:
if os.path.exists(feature_extracted):
    ft = pd.read_csv(feature_extracted, delimiter=',', header=0)
else:
    ft = feature_extraction(df, sample_rate, step, pca_tolerance, undersampling_rate, oversampling_neighbors)

ft

Train and Test process

In [ ]:
x_train, x_test, y_train, y_test, results, kf = trainTestData (ft, test_ratio, k_fold, perfInd)

x_train = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
y_train = y_train.values.astype(int)
x_test = np.reshape(x_test.values, (x_test.shape[0], 1, x_test.shape[1]))
y_test = y_test.values.astype(int)

In [ ]:
k_fold = 5
x_train, x_test, y_train, y_test, kf = trainTestData_k_fold (ft, test_ratio, k_fold)

In [ ]:
x_train, x_test, y_train, y_test, x_val, y_val = trainTestData_validation (ft, test_ratio, val_ratio)

CNN

In [ ]:
def CNN():

    model = Sequential()

    # C1
    model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(22, 59, 114)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C2
    model.add(Conv2D(32, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C3
    model.add(Conv2D(64, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))

    return model

def CNN_dropout():

    model = Sequential()

    # C1
    model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(22, 59, 114)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # C2
    model.add(Conv2D(32, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # C3
    model.add(Conv2D(64, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))

    return model

def CNN_GRU():

    model = Sequential()

    # C1
    model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(22, 59, 114)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C2
    model.add(Conv2D(32, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C3
    model.add(Conv2D(64, (2, 2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='sigmoid'))

    # Reshape layer
    model.add(Reshape((16, 16)))

    # GRU
    model.add(GRU(128))

    model.add(Dense(2, activation='softmax'))

    return model

def CNN_LSTM():
    
    model = Sequential()
    # C1
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(22, 59, 114)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C2
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # C3
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(TimeDistributed(Flatten()))

    # LSTM
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))

    return model

In [ ]:
init()

if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'Model1.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=save_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True)
]

model = CNN()

optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Recall(name='sen')])
    
model.summary()

history = model.fit(generate_arrays_for_training(filesPath, end=80),
                        validation_data=generate_arrays_for_training(filesPath, start=20),
                        steps_per_epoch=150,
                        validation_steps=50,
                        epochs=100, max_queue_size=2, shuffle=True, callbacks=[callbacks_list])

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Creates a HDF5 file 
model.save("ModelOutSeizure3.h5")
print("Model saved")
        
plot_performance(train_loss, val_loss, train_acc, val_acc)

print('Training end')

print('Evaluating Model')


In [ ]:
init()
model = load_model('model_checkpoint/CNN.h5')
print('Evaluating Model')
evaluation_model(model)

In [ ]:
init()

if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'Model2.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=save_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True)
]

model = createModel2()

optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Recall(name='sen')])
    
model.summary()

history = model.fit(generate_arrays_for_training(filesPath, end=75), #end=75),#It take the first 75%
                        validation_data=generate_arrays_for_training(filesPath, start=75),#start=75), #It take the last 25%
                        steps_per_epoch=150, 
                        validation_steps=50,
                        epochs=100, max_queue_size=2, shuffle=True, callbacks=[callbacks_list])# 100 epochs è meglio #aggiungere criterio di stop in base accuratezza

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
        
plot_performance(train_loss, val_loss, train_acc, val_acc)

print('Training end')

In [ ]:
init()
model = load_model('model_checkpoint/CNN_Dropout.h5')
print('Evaluating Model')
evaluation_model(model)

In [ ]:
init()

if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'Model3.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=save_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True)
]

model = CNN_GRU()

optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Recall(name='sen')])
    
model.summary()

history = model.fit(generate_arrays_for_training(filesPath, end=75),
                        validation_data=generate_arrays_for_training(filesPath, start=75),
                        steps_per_epoch=100,
                        validation_steps=50,
                        epochs=75, max_queue_size=2, shuffle=True, callbacks=[callbacks_list])

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plot_performance(train_loss, val_loss, train_acc, val_acc)

print('Training end')

In [ ]:
init()
model = load_model('model_checkpoint/CGRUNN.h5')
print('Evaluating Model')
evaluation_model(model)

In [ ]:
init()

if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'Model4_1.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=save_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True)
]
model = CNN_LSTM()

optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Recall(name='sen')])
    
model.summary()

history = model.fit(generate_arrays_for_training(filesPath, end=75),
                        validation_data=generate_arrays_for_training(filesPath, start=75),
                        steps_per_epoch=100,
                        validation_steps=50,
                        epochs=200, max_queue_size=2, shuffle=True, callbacks=[callbacks_list])

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plot_performance(train_loss, val_loss, train_acc, val_acc)

print('Training end')


In [ ]:
init()
model = load_model('model_checkpoint/CNN_LSTM.h5')
print('Evaluating Model')
evaluation_model(model)

RNN

In [ ]:
dir_name = 'model_checkpoint'

if not os.path.exists(dir_name):
        os.makedirs(dir_name)

save_path = os.path.join(dir_name, 'Vanilla_RNN.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

plt.figure()
for i in range(5):
    plt.plot(train_loss[i])
plt.title('Training Loss - All Folds')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()
for i in range(5):
    plt.plot(train_acc[i])
plt.title('Training Accuracy - All Folds')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

GRU

In [ ]:
# This code illustrates the implementation of an enhanced GRU model with a single layer and Stochastic Gradient Descent optimization. 
# The model is trained to solve a binary classification problem

dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_1 layer_SGD.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32, input_shape=(None, x_train.shape[-1])))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with a SGD optimizer with an exponential decaying learning rate
optimizer, lr_schedule = optimizer_SGD(0.001, 1000, 0.1)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=200, validation_data=(x_val, y_val), callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule),callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



In [ ]:
dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_1 layer_Adam.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32, input_shape=(None, x_train.shape[-1])))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer
optimizer = optimizers.Adam(learning_rate=0.0001)  
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=200, validation_data=(x_val, y_val), callbacks=[callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_1 layer_RMSprop.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32, input_shape=(None, x_train.shape[-1])))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with RMSprop optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001) 
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=200, validation_data=(x_val, y_val), callbacks=[callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# This code illustrates the implementation of an improved GRU model with dropout regularization and Adam optimization.

dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_1 layer with dropout_Adam.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32,input_shape=(None, x_train.shape[-1])))
model.add(Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer
optimizer = optimizers.Adam(learning_rate=0.0001)  
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=200, validation_data=(x_val, y_val), callbacks=[callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# This code illustrates the implementation of an improved GRU model with dropout regularization and Root Mean Square Propagation optimization.

dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_1 layer with dropout_RMSprop.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32,input_shape=(None, x_train.shape[-1])))
model.add(Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with RMSprop optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)  
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=200, validation_data=(x_val, y_val), callbacks=[callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# This code implements a deep Gated Recurrent Unit (GRU) architecture with two layers, using the SGD optimization.


dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_2 layers_SGD.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(layers.GRU(32, input_shape=(None, x_train.shape[-1]),return_sequences=True))
model.add(layers.GRU(32))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with a SGD optimizer with an exponential decaying learning rate
optimizer, lr_schedule = optimizer_SGD(0.001, 1000, 0.1)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=5, epochs=700, validation_data=(x_val, y_val), callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), callbacks_list])


plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# This code implements a K-Fold Cross Validation technique combined with a deep Gated Recurrent Unit (GRU) architecture. 

dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'GRU_2 layers k fold.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

k_fold = 5 # number of folds for the K-fold cross validation
x_train, x_test, y_train, y_test, kf = trainTestData_k_fold (ft, test_ratio, k_fold)

# Arrays to store the learning curves at each k-th iteration
val_loss = []
val_acc = []
train_loss = []
train_acc = []
test_loss = []
test_acc = []

print('Implementing GRU with K-fold')
start = time.time()
for train, test in kf.split(ft):
    x_train = ft.iloc[train,:ft.shape[1]-1]
    x_train = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
    y_train = ft.loc[train,'seizure'].values.astype(int)
    x_test = ft.iloc[test,:ft.shape[1]-1]
    x_test = np.reshape(x_test.values, (x_test.shape[0], 1, x_test.shape[1]))
    y_test = ft.loc[test,'seizure'].values.astype(int)

    # Definition of the model
    model = Sequential()
    model.add(layers.GRU(32, input_shape=(None, x_train.shape[-1]),return_sequences=True))
    model.add(layers.GRU(32))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model with a SGD optimizer with an exponential decaying learning rate
    optimizer, lr_schedule = optimizer_SGD(0.001, 1000, 0.1)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Training of the model
    history = model.fit(x_train, y_train, batch_size = 5, epochs = 700, verbose = 0, validation_data=(x_test,y_test), callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), callbacks_list])

    # Store the metrics values for each epoch and for each fold
    train_loss.append(history.history['loss'])
    train_acc.append(history.history['accuracy'])
    val_loss.append(history.history['val_loss'])
    val_acc.append(history.history['val_accuracy'])

    # Evaluation of the model
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    test_acc.append(accuracy)
    test_loss.append(loss)

    # Print of the loss and accuracy scores at the end of each fold
    print("Loss: {:.4f}, Accuracy: {:.2f}%".format(loss, accuracy * 100))

end = time.time()
t = round(end - start,2)
print('GRU finished in', t,'sec\n')

# Plot of the average learning curves
plot_k_fold(train_loss, train_acc, val_loss, val_acc)

# Calculate average performance
avg_accuracy = np.mean(test_acc)
avg_loss = np.mean(test_loss)
print(f'Average accuracy: {avg_accuracy:.4f}')
print(f'Average loss: {avg_loss:.4f}')

LSTM + GRU

In [ ]:
dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'LSTM + GRU_SGD.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

# Definition of the model
model = Sequential()
model.add(LSTM(32, return_sequences=True,input_shape=(None, x_train.shape[-1])))
model.add(layers.GRU(32))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a SGD optimizer with an exponential decaying learning rate
optimizer, lr_schedule = optimizer_SGD(0.001, 1000, 0.1)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training of the model
history = model.fit(x_train, y_train, batch_size=6, epochs=1000, validation_data=(x_val, y_val), callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule),callbacks_list])

plot_validation(history)

# Evaluation of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
dir_name = 'model_checkpoint'
if not os.path.exists(dir_name):
        os.makedirs(dir_name)
save_path = os.path.join(dir_name, 'LSTM + GRU_10-fold.h5')

callbacks_list = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, monitor="val_loss", verbose=1, save_best_only=True)

k_fold = 10 # number of folds for the K-fold cross validation
x_train, x_test, y_train, y_test, kf = trainTestData_k_fold (ft, test_ratio, k_fold)

# Arrays to store the learning curves at each k-th iteration
val_loss = []
val_acc = []
train_loss = []
train_acc = []
test_loss = []
test_acc = []

print('Implementing GRU with K-fold')
start = time.time()
for train, test in kf.split(ft):
    x_train = ft.iloc[train,:ft.shape[1]-1]
    x_train = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
    y_train = ft.loc[train,'seizure'].values.astype(int)
    x_test = ft.iloc[test,:ft.shape[1]-1]
    x_test = np.reshape(x_test.values, (x_test.shape[0], 1, x_test.shape[1]))
    y_test = ft.loc[test,'seizure'].values.astype(int)

    # Definition of the model
    model = Sequential()
    model.add(LSTM(64, return_sequences=True,input_shape=(None, x_train.shape[-1])))
    model.add(layers.GRU(32))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with a SGD optimizer with an exponential decaying learning rate
    optimizer, lr_schedule = optimizer_SGD(0.001, 1000, 0.1)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Training of the model
    history = model.fit(x_train, y_train, batch_size = 5, epochs = 700, verbose = 0, validation_data=(x_test,y_test), callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), callbacks_list])

    # Store the metrics values for each epoch and for each fold
    train_loss.append(history.history['loss'])
    train_acc.append(history.history['accuracy'])
    val_loss.append(history.history['val_loss'])
    val_acc.append(history.history['val_accuracy'])

    # Evaluation of the model
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    test_acc.append(accuracy)
    test_loss.append(loss)

    # Print of the loss and accuracy scores at the end of each fold
    print("Loss: {:.4f}, Accuracy: {:.2f}%".format(loss, accuracy * 100))

end = time.time()
t = round(end - start,2)
print('Vanilla_RNN finished in', t,'sec\n')

# Plot of the average learning curves
plot_k_fold(train_loss, train_acc, val_loss, val_acc)

# Calculate average performance
avg_accuracy = np.mean(test_acc)
avg_loss = np.mean(test_loss)
print(f'Average accuracy: {avg_accuracy:.4f}')
print(f'Average loss: {avg_loss:.4f}')